In [1]:
import pandas as pd
import numpy as np
from predictables.core.src._UnivariateAnalysis import UnivariateAnalysis
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the breast cancer dataset
bcancer = load_breast_cancer()
cancery = pd.Series(bcancer.target, name="y").map({0: "malignant", 1: "benign"})
cancerdf = pd.DataFrame(
    bcancer.data, columns=[c.replace(" ", "_") for c in bcancer.feature_names]
)

# Standardize the data
scaler = StandardScaler()
cancerdf = pd.DataFrame(scaler.fit_transform(cancerdf), columns=cancerdf.columns)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    cancerdf, cancery, test_size=0.2, random_state=42, stratify=cancery
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train
)

# Perform PCA
pca = PCA(n_components=2)
X_train_pca = pd.DataFrame(
    pca.fit_transform(X_train), columns=["PC1", "PC2"], index=X_train.index
)
X_val_pca = pd.DataFrame(
    pca.transform(X_val), columns=["PC1", "PC2"], index=X_val.index
)
X_test_pca = pd.DataFrame(
    pca.transform(X_test), columns=["PC1", "PC2"], index=X_test.index
)

# Combine the data
df_train = pd.concat([X_train, X_train_pca], axis=1)
df_val = pd.concat([X_val, X_val_pca], axis=1)
df_test = pd.concat([X_test, X_test_pca], axis=1)

# Add the target variable
df_train["y"] = y_train.map({"malignant": 0, "benign": 1}).values
df_val["y"] = y_val.map({"malignant": 0, "benign": 1}).values
df_test["y"] = y_test.map({"malignant": 0, "benign": 1}).values

# Randomly sort training data into 5 cross-validation folds
df_train["fold"] = np.random.choice(range(5), size=df_train.shape[0]) + 1

df_train.to_parquet("cancer_train.parquet")
df_val.to_parquet("cancer_val.parquet")
df_test.to_parquet("cancer_test.parquet")

df_train.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,PC1,PC2,y,fold
194,0.208100,0.912292,0.347273,0.046959,0.572150,1.774977,1.015706,1.028170,-0.272428,0.556040,...,-0.033692,1.339296,0.895753,0.884571,0.160555,0.169804,2.601883,1.664731,0,2
46,-1.684571,-0.570050,-1.658278,-1.288347,-0.737294,-0.851130,-0.915500,-1.109197,-0.155598,0.316465,...,-0.116980,-0.754239,-0.975761,-1.354653,0.330422,-0.546168,-4.564517,1.565109,1,5
332,-0.825712,0.132725,-0.825000,-0.761051,0.643316,-0.692695,-1.052023,-1.066224,0.468713,-0.356897,...,0.439736,-1.002397,-1.241784,-1.437181,0.632947,-1.037706,-3.127716,1.357112,1,1
76,-0.169639,-1.943019,-0.167192,-0.272150,2.329937,0.006804,-0.251467,0.429234,2.159100,0.512094,...,0.558093,-0.740244,-0.896170,-0.617229,-0.308601,-0.666975,-0.807603,2.071959,1,1
124,-0.215082,-0.674768,-0.241747,-0.288361,-1.794101,-0.589220,-0.098925,-0.539588,-1.422476,-0.647506,...,-1.309316,-0.007411,0.281190,-0.378019,-1.379572,-0.424808,-2.212381,-0.936765,1,5


In [2]:
ua = UnivariateAnalysis(
    model_name="Cancer Model",
    df_train=df_train,
    df_val=df_val,
    target_column_name="y",
    feature_column_names=df_train.drop(columns=["y", "fold"]).columns.tolist(),
    cv_column_name="fold",
    has_time_series_structure=False,
)

Performing univariate analysis on 32 features:   0%|          | 0/32 [00:00<?, ?it/s]

In [7]:
ua.pc1.agg_results.collect()

fold
str
null
null
null
null
null
"""mean"""
"""std"""


In [3]:
ua.build_report("credit.pdf", max_per_file=20)

ColumnNotFoundError: feature

Error originated just after this operation:
DF ["fold"]; PROJECT */1 COLUMNS; SELECTION: "None"

Error originated just after this operation:
ErrorStateSync(AlreadyEncountered(not found: feature

Error originated just after this operation:
DF ["fold"]; PROJECT */1 COLUMNS; SELECTION: "None"))
DF ["fold"]; PROJECT */1 COLUMNS; SELECTION: "None"

In [ ]:
import polars as pl


def results_tbl_expr(col, formatted_name, formatting_str) -> pl.Expr:
    """
    Create an expression to format a column in a results table.

    Parameters
    ----------
    col : pl.Expr
        The column to format.
    formatted_name : str
        The name of the formatted column.
    formatting_str : str
        The formatting string.

    Returns
    -------
    pl.Expr
        The formatted column expression.
    """
    return pl.format(formatting_str, pl.col(col)).cast(pl.Utf8).alias(formatted_name)

SyntaxError: f-string: expression required before ':' (771779236.py, line 25)

In [ ]:
ua._sort_features_by_ua()

TypeError: df must be a pandas or polars dataframe. Got <class 'str'>.

### fit each fold to a model

In [ ]:
X, y = (
    df.collect().to_pandas().drop(columns="target"),
    df.select("target").collect().to_pandas()["target"],
)
X.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,cv
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,2
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,5
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,3
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [ ]:
df = df.collect().to_pandas()
df.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,target,cv
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0,2
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0,5
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0,3
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0,1


In [ ]:
from predictables.univariate.src._fit_sm_logistic_regression import (
    fit_sm_logistic_regression,
)
from predictables.core._PredicTables import PredicTables
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

fit_sm_logistic_regression(X_train.iloc[:, :3], y_train)

In [ ]:
pt = PredicTables(
    "Cancer Model",
    pd.concat([X_train.iloc[:, :3], y_train], axis=1),
    pd.concat([X_test.iloc[:, :3], y_test], axis=1),
    pd.concat([X_test.iloc[:, :3], y_test], axis=1),
    "target",
    X_train["cv"],
    False,
)
pt.ua.build_report("new_test_report.pdf", max_per_file=5)

TypeError: UnivariateAnalysis.__init__() got multiple values for argument 'has_time_series_structure'

In [ ]:
pt.ua._sort_features_by_ua().index.tolist()

Sorting features by their relevance based on an univariate analysis: 100%|██████████| 3/3 [00:00<00:00, 17403.75it/s]


['mean_texture', 'mean_radius', 'mean_perimeter']

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

lr = LinearRegression().fit(np.array([1, 2, 3]).reshape(-1, 1), np.array([1, 2, 3]))
lr2 = sm.GLM(
    np.array([1, 2, 3]), np.array([1, 0, 1]), family=sm.families.Binomial()
).fit()

dir(lr)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_decision_function',
 '_estimator_type',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_set_intercept',
 '_validate_data',
 '_validate_params',
 'coef_',
 'copy_X',
 'fit',
 'fit_intercept',
 'get_metadata_routing',
 'get_params',
 'intercept_',
 'n_features_in_',
 'n_jobs',
 'positive',
 

In [ ]:
dir(lr2)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_data_attr',
 '_data_attr_model',
 '_data_in_cache',
 '_endog',
 '_freq_weights',
 '_get_robustcov_results',
 '_get_wald_nonlinear',
 '_iweights',
 '_n_trials',
 '_transform_predict_exog',
 '_use_t',
 '_var_weights',
 'aic',
 'bic',
 'bic_deviance',
 'bic_llf',
 'bse',
 'conf_int',
 'converged',
 'cov_kwds',
 'cov_params',
 'cov_type',
 'deviance',
 'df_model',
 'df_resid',
 'f_test',
 'family',
 'fit_history',
 'fittedvalues',
 'get_distribution',
 'get_hat_matrix_diag',
 'get_influence',
 'get_prediction',
 'info_criteria',
 'initialize',
 'k_constant',
 'llf',
 'llf_scaled',
 'llnull',
 'lo

In [ ]:
pd.__version__

'2.1.2'

In [ ]:
lr.n_features_in_

1